<a href="https://colab.research.google.com/github/yjyjy131/Study_Deep_Learning/blob/main/Deep-learning-from-scratch/chapter_4_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 미니배치 학습 구현하기

In [6]:
import numpy as np

In [27]:
from functions import *
from gradient import numerical_gradient
from mnist import load_mnist

In [28]:
class TestNet:
  def __init__(self, in_size, hidden_size, out_size, weight=0.01) :
    self.params = {}
    self.params['W1'] = weight * np.random.randn(in_size, hidden_size)
    self.params['b1'] = np.zeros(hidden_size)
    self.params['W2'] = weight * np.random.randn(hidden_size, out_size)
    self.params['b2'] = np.zeros(out_size)

  
  def predict(self, x) :
    W1, W2 = self.params['W1'], self.params['W2']
    b1, b2 = self.params['b1'], self.params['b2']

    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    y = softmax(a2)

    return y


  def loss(self, x, t) :
    y = self.predict(x)
    return cross_entropy_error(y, t)


  def accuracy(self, x, t) :
    y = self.predict(x)
    y = np.argmax(y, axis=1)
    t = np.argmax(t, axis=1)

    acc = np.sum(y==t) / float(x.shape[0])
    return acc


  # gradient 구현 함수는 생략되어있음
  def numerical_gradient(self, x, t) :
    loss_W = lambda W : self.loss(x, t)

    grads = {}
    grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
    grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
    grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
    grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

    return grads

In [ ]:
(x_train, y_train), (x_test, y_test) = load_mnist(normalize=True, one_hot_label=True)

train_loss = []
train_acc = []
test_acc = []

epochs = 10000
train_size = x_train.shape[0]
batch_size = 100
lr = 0.1
net = TestNet(in_size=784, hidden_size=50, out_size=10)

# 나누기 후 소수 1번째자리까지
iter_per_epoch = max(train_size/batch_size, 1)

In [ ]:
for i in range(epochs) :
  # train_size 만큼의 숫자에서 랜덤으로 batch_size 개수의 숫자를 뽑아냄
  batch_mask = np.random.choice(train_size, batch_size)

  x_batch = x_train[batch_mask]
  y_batch = y_train[batch_mask]

  grad = net.numerical_gradient(x_batch, y_batch)

  for k in ('W1', 'b1', 'W2', 'b2') :
    net.params[k] -= lr * grad[k]

  loss = net.loss(x_batch, y_batch)
  train_loss.append(loss)

  if i % iter_per_epoch == 0 :
    tmp_train_acc = net.accuracy(x_train, y_train)
    tmp_test_acc = net.accuracy(x_test, y_test)
    train_acc.append(tmp_train_acc)
    test_acc.append(tmp_test_acc)
    print(f'train acc : {tmp_train_acc} / test acc : {tmp_test_acc}') 